In [2]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tweepy import TweepError

pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

In [3]:
consumer_key = "Y9LSrr5Ctyco96MxuB5lVrXLE" # Use your own key. To get a key https://apps.twitter.com/
consumer_secret = "MZ0ZzvzcyDFQAri4YFQ1NUJIUdyaaKr22YoHt1sdE6yVkUPOk4"

access_token = "1718965579-EbDVvYloIQWEMdrC62fjSDHnkmjW3a876S33G7h"
access_token_secret = "fxxW6evy2Gzlxc0k16d4Lckpkd2QQXW2obXFGZh5divHe"

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [4]:
f = open('applesuckstweets-v2.0.html')
html_doc = "\n".join(f.readlines())
soup = BeautifulSoup(html_doc, 'html.parser')

# html_doc = soup.prettify()
# soup = BeautifulSoup(html_doc, 'html.parser')

In [5]:
links = soup.findAll('a')

extracted_tweets = []
extracted_tweets_ids = set()
for link in links:
    # print link.attrs
    # print type(link)
    # print link
    if link.has_attr('href'):
        href = link['href']
    # print href
    if 'status' in href:
        status_id = href.split("/")[-1]
        print "%4d"%len(extracted_tweets_ids), status_id, href
        
        if status_id.isdigit():
            try:
                tweet = api.get_status(status_id)
                extracted_tweets.append(tweet)
                extracted_tweets_ids.add(status_id)
            except TweepError as e:
                print e

798173858410983425 /Tedescop129/status/798173858410983425
798171734918787073 /Tedescop129/status/798171734918787073
798027051198545921 /JordanaSays/status/798027051198545921
797938783840653312 /emkaraokes/status/797938783840653312
797876478843047936 /LaurenStallone/status/797876478843047936
797710688022069248 /That_AndLight/status/797710688022069248
797070427235360768 /DavidWreathall/status/797070427235360768
796636944277078016 /ma30ous/status/796636944277078016
796474741376880642 /chellegbk/status/796474741376880642
796450925405536256 /leshka1992/status/796450925405536256
796252849478844416 /ravioleey/status/796252849478844416
796110407253360640 /stylatarium/status/796110407253360640
795796706348634112 /kylababy912/status/795796706348634112
795717718922199045 /Garo_AE/status/795717718922199045
795705607697395712 /mtlangan/status/795705607697395712
795179651039686656 /Darkjediken323/status/795179651039686656
795110891549298688 /MarcusTurbo/status/795110891549298688
795098251917524992 /

In [7]:
print len(extracted_tweets)
print len(extracted_tweets_ids)

701
700


In [13]:
def process_tweets(tweets):
    id_list = [tweet.id for tweet in tweets]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in tweets]
    data_set["created_at"] = [tweet.created_at for tweet in tweets]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in tweets]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in tweets]
    data_set["is_replay_to"] = [tweet.in_reply_to_status_id is not None for tweet in tweets]
    data_set["source"] = [tweet.source for tweet in tweets]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in tweets]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in tweets]
    data_set["user_name"] = [tweet.author.name for tweet in tweets]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in tweets]
    data_set["user_description"] = [tweet.author.description for tweet in tweets]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in tweets]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in tweets]
    data_set["favourites_count"] = [tweet.author.favourites_count for tweet in tweets]
    data_set["statuses_count"] = [tweet.author.statuses_count for tweet in tweets]
    data_set["user_location"] = [tweet.author.location for tweet in tweets]

    return data_set

data_set = process_tweets(extracted_tweets)

In [14]:
data_set.head(1)

,id,text,created_at,retweet_count,favorite_count,is_replay_to,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,favourites_count,statuses_count,user_location
0,798173858410983425,Best news yet! #makeamericagreatagain #trump #...,2016-11-14 14:40:51,0,1,False,Instagram,1659941960,Tedescop129,Pasquale Tedesco,2013-08-10 11:49:33,,107,277,31,1109,


In [15]:
import openpyxl
from pandas import ExcelWriter

writer = ExcelWriter('tweets.xlsx')
data_set.to_excel(writer,'Sheet1')
writer.save()